In [ ]:
from fit_data_1_22_common import *
from fit_data_1_22_list_maskers import *

pl.style.use('fivethirtyeight')

#mpl.rc('figure', figsize=(10,8))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from optim import *

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Common for all CFs

In [ ]:
#see fit_data_1_22_common.py

#plot_main_CAPs()
#plot_CAP_with_window()
#plot_CAP_w_wo_filter()

#NB: the plots below depend on the choice of the signal used for the estimation of ur
#plot_raw_excitation_deconv()
#plot_figures_narrowband_analysis()
#plot_figures_narrowband_analysis_deconv()

#plot_estimated_latencies_deconv()
#plot_estimated_latencies_deconv()
#plot_latencies_fit()

### CF specific

first estimation I/O curve

In [ ]:
#CF=8000
CF=4000

In [ ]:

cap=[]
rms=[]
masker_list=ntch_masker_lists[CF]  #, 'broadband_noise'
for masker in masker_list:
    sig=capData.get_signal_by_name(masker)
    sig=process_signal(sig)
    broadband_sig_trunc=process_signal(broadband2)
    
    #REF broadband
    cap.append(np.max(sig-broadband_sig_trunc)-np.min(sig-broadband_sig_trunc))
    #rms.append(np.std(sig-broadband_sig_trunc))
    
    #cap.append(np.max(sig)-np.min(sig))
    #rms.append(np.std(sig-broadband_sig_trunc))
    pl.plot(t2, sig-broadband_sig_trunc, label=masker)


pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.show()

pl.figure(figsize=(8, 6))

attns=-np.array([55, 50, 45,40,35,32,29,26,23])   # 20])
pl.plot(attns, cap, '+', label='max-min')

#pl.plot(-np.array([55,50,45,40,37,34,31,28,25,22]), np.array(rms)*10, label='rms x10')
pl.legend()
pl.xlabel('Notch attenuation')
pl.ylabel('Amplitude difference')


In [ ]:
sigm=SigmoidIOFunc(0, 0)
#maskamount=1-(  (cap-np.amin(cap)) /np.amax(cap-np.amin(cap)) )
maskamount=1-(cap/np.amax(cap)) 

I_pts=I0+attns
#sigm.fit_data(I_pts, maskamount, set_mmax=True)

#HACK enforce masking=100% at attn20
#sigm.mmax.data*=1/sigm(I0-20)

sigm.fit_data(I_pts, maskamount, constrained_at_Iref=True, Iref=I0-20)

I=np.linspace(-30, 20)
pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)

pl.plot(I0-20, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (max: broadband)')

In [ ]:
wb_cdf=WeibullCDF_IOFunc()

wb_cdf.fit_data(I_pts, maskamount, constrained_at_Iref=True, Iref=I0-20)

In [ ]:
pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')
pl.plot(I, wb_cdf(torch.tensor(I)), label='fit sigm')
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)

pl.plot(I0-20, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (max: broadband)')

Setting model

In [ ]:
ntch_maskerNames, ntch_maskingConds, ntch_signals =capData.get_batch_re(ntch_regexps[CF])
ntch_maskingConds.set_amp0_dB(I0)

In [ ]:
#HACK pad maskers >12e3 to avoid issues with latencies (equivalent to taking the difference
#  excitations of maskers - excitation 'broadband noise')
ntch_maskingConds.pad_maskers(f_thr=11000, f_max=1e5)

In [ ]:
gauss_sigma=(1.5e-4)/(t2[1]-t2[0])
ntch_signals_proc=process_signal2(ntch_signals, gauss_sigma=gauss_sigma)

In [ ]:
# test

for maskerName, sig in zip(ntch_maskerNames, ntch_signals_proc):
    pl.plot(t2, sig, label=maskerName)

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# estimation ur

sig=capData.get_signal_by_name('4_notch8000_bw2300_45dB')
sig2=process_signal(sig)

pl.plot((t2-3e-3)*1e3, sig2-broadband_proc)

#pl.plot(t2*1e3, sig2bis-broadband_proc)

t_shift=6e-3-3e-3 #excitation coincides with CM

ur0=sig2-broadband_proc
ur0=np.roll(ur0,  -int(t_shift*48828) )
pl.plot((t2-3e-3)*1e3, ur0)
#pl.xlim([4,6])

In [ ]:
#latencies model

#lat_model=lat_above4k
#lat_model=lat

#lat_model=PowerLawLatencies.shift(lat_model, 6e-3-1e-3)   #t0: start CM-1ms

#HACK as latencies are very small (sampling issues), manual dilatation

lat_model=PowerLawLatencies.fromPts(0.0057, 9500, 0.0062, 6000)

In [ ]:
#test single lat model
singleLat=True
if singleLat:
    if CF>6500:
        f_min=4000
        f_max=12000
    elif CF < 5500:
        if CF<4500:
            f_min=2000
            f_max=7500
        else:
            f_min=2200
            f_max=8000
    else:
        f_min=2500
        f_max=9000
    lat_model = SingleLatency(6e-3, f_min=f_min, f_max=f_max)


In [ ]:
if singleLat:
    m=400
    E0=1/2*np.ones((m,))
    
    pl.plot(np.linspace(f_min*1e-3, f_max*1e-3, m), E0)
    pl.xlabel('Frequency (kHz)')
    pl.ylabel('Init raw excitation')
else:
    m=72
    E0_temp=sg.windows.tukey(m, alpha=0.5) 
    E0=np.zeros_like(t2)
    ind_begin=int((t_shift-1e-3)*48828)
    ind_end=int((t_shift-1e-3)*48828)+m
    E0[ind_begin:ind_end]=E0_temp

    pl.plot(t2*1e3, E0)
    pl.title('Init raw excitation')
    pl.xlabel('t (ms)')
    pl.ylabel('Amp')

In [ ]:
plotLatencies(lat_model)

In [ ]:
E=ExcitationPatterns(t2, E0)  #no non-maskable part
#E=ExcitationPatterns(t2, E_temp.E0_maskable)

BW10_0=2000
BW10_0Func=constant_BW10(BW10_0, requires_grad=False)

#E.set_masking_model(lat_model, BW10_0Func, ntch_maskingConds, sigm)
E.set_masking_model(lat_model, BW10_0Func, ntch_maskingConds, wb_cdf)

In [ ]:
#estimation ur

maskAmounts, excs = E.get_tensors() 

nb_steps=20
alpha=np.linspace(0.5, 0.05, nb_steps)

EPs_fft=np.fft.rfft(excs, axis=1)
CAPs_fft=np.fft.rfft(ntch_signals_proc, axis=1)
#u1_mat=np.tile(ur0, (ntch_maskingConds.n_conditions, 1))
u1_mat=np.zeros_like(ntch_signals_proc)
filter_mat  = (t2>7.5e-3)+(t2<3.2e-3)
filter_mat=np.tile(filter_mat, (ntch_maskingConds.n_conditions, 1))
#filter_mat=np.zeros_like(ntch_signals_proc, dtype=bool)
#proj_fft=E.get_projector_fft()

weights=np.sqrt(np.sum(excs.clone().detach().numpy()**2, axis=1))
for i in range(1, nb_steps+1):
    du=deconv_newton_step(u1_mat, EPs_fft, CAPs_fft, eps_ridge=0)   #TODO proj_fft
    #du=deconv_grad(u1_mat, EPs_fft, CAPs_fft)
    
    u1_mat-=alpha[i-1]*du
    #proj 1 
    u1_mat[filter_mat]=np.zeros_like(u1_mat[filter_mat])
    #proj 2

    #u1_mat_mean=np.mean(u1_mat, axis=0)[None, :]
    
    
    #HACK waiting for proj_fft
    u1_mat_mean=np.average(u1_mat, axis=0, weights=weights)[None, :]
    u1_mat=np.repeat(u1_mat_mean, ntch_maskingConds.n_conditions, axis=0)
     
    '''
    for i in range(5):
            pl.figure()
            name=ntch_maskerNames[i]
            pl.plot(u1_mat[i], label=name, color=f'C{i}')
            #pl.plot( np.abs(EPs_fft[i]))
            pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    '''
    
    if i==nb_steps:
        pl.figure()
        pl.title(f'Step {i} (deconv + proj)')
        #pl.plot(t, u0, label='u0 (truth)')
        pl.plot(t2, u1_mat[0], label='u0 (estimated)')
        pl.legend()
        #pl.savefig('ur_8kHz_Q395.svg')
        pl.show()

Test estimation raw excitation pattern

In [ ]:
fln_list=vfreq_fln_lists[CF]
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]

reg_exp=')|('.join(masker_list)
reg_exp='('+reg_exp+')'
vfreq_maskerNames, vfreq_maskingConds, vfreq_signals =capData.get_batch_re(reg_exp)
vfreq_signals_proc=process_signal2(vfreq_signals, gauss_sigma=gauss_sigma)
vfreq_maskingConds.set_amp0_dB(I0)
vfreq_maskingConds.pad_maskers(f_thr=11000, f_max=1e5)



In [ ]:
signals_proc=vfreq_signals_proc
maskingConds=vfreq_maskingConds

E_temp=ExcitationPatterns.copyRaw(E, requires_grad=True)

#E_temp.set_masking_model(lat_model, BW10_0Func, maskingConds, sigm)
E.set_masking_model(lat_model, BW10_0Func, maskingConds, wb_cdf)


In [ ]:

nb_steps=30
alpha=20
n_dim=7 #projection of gradient on n_dim first dimensions (Fourier basis)

alpha_dic={E_temp.E0_maskable: alpha}
optim_steps(E_temp, u1_mat[0], signals_proc, alpha_dic, 
            nb_steps=nb_steps, n_dim_E0=n_dim, plot_E0_graph=True, step_plots=5 )


In [ ]:
E.E0_maskable.data=E_temp.E0_maskable.data

pl.figure(figsize=(10,20))
plotExcitationPatterns(E, plot_raw_excitation=True) # ylim_top=1
pl.show()

test model before fine-tuning I/0 curve

fine-tuning I/O curve

In [ ]:
# try more accurate estimation of i/o curve

signals_proc=ntch_signals_proc
maskingConds=ntch_maskingConds


io_func = 'weibull'  #'sigm'
sigm2=SigmoidIOFunc(sigm.mu.numpy(), sigm.a.numpy(), Iref=I0-20, constrained_at_Iref=True, requires_grad=True)


# wb_cdf2=WeibullCDF_IOFunc(I0=wb_cdf.I0.numpy(),
#     scale=wb_cdf.scale.numpy(),
#     mmax=wb_cdf.mmax.numpy(),
#     k=wb_cdf.k.numpy(),
#     requires_grad=True,
#     constrained_at_Iref=True,
#     Iref=I0-20)

wb_cdf2=WeibullCDF_IOFunc(I0=-20.,
    scale=30.,
    k=10.,
    mmax=1.,
    requires_grad=True,
    constrained_at_Iref=True,
    Iref=I0-20)

#E2=ExcitationPatterns(t2, E0, requires_grad=True)  #no non-maskable part
E2=ExcitationPatterns.copyRaw(E, requires_grad=True)
BW10_0TestFunc=constant_BW10(2000., requires_grad=False)

if io_func=='weibull':
    E2.set_masking_model(lat_model, BW10_0TestFunc, ntch_maskingConds, wb_cdf2)
else:
    E2.set_masking_model(lat_model, BW10_0TestFunc, ntch_maskingConds, sigm2)

In [ ]:
nb_steps=100
alpha=10


if io_func=='weibull':
    alpha_dic={wb_cdf2.I0: alpha, wb_cdf2.scale: alpha, wb_cdf2.k: alpha}
else:
    alpha_dic={sigm2.mu: alpha, sigm2.a: alpha}

#alpha_dic[BW10_0TestFunc.BW_10]=alpha
alpha_dic[E2.E0_maskable]=alpha  #/!| with sum_grad_E0 set to True
    

step_verbose=20




optim_steps(E2, u1_mat[0], signals_proc, alpha_dic, 
            nb_steps=nb_steps, sum_grad_E0=True, 
            plot_E0_graph=True, plot_masking_I0_graph=True,
            #plot_Q10=True, fc_ref_Q10=CF,
           step_plots=5)



In [ ]:
#model
with torch.no_grad():
    u1=u1_mat[0]
    pl.figure(figsize=(12,20))
    ax_list=plotSimulatedCAPs(E2, u1, ylim=[-0.01, 0.01], max_plots=10)
    plotSimulatedCAPs(E2, CAParray=ntch_signals_proc, axlist=ax_list, max_plots=10)
    pl.plot()

In [ ]:
fln_list=vbw_fln_lists[CF]
masker_list=[st.replace('-', '_').replace('.json', '') for st in fln_list]
reg_exp=')|('.join(masker_list)
reg_exp='('+reg_exp+')'
vbw_maskerNames, vbw_maskingConds, vbw_signals =capData.get_batch_re(reg_exp)
vbw_signals_proc=process_signal2(vbw_signals, gauss_sigma=gauss_sigma)
vbw_maskingConds.set_amp0_dB(I0)

In [ ]:
#HACK
vbw_maskingConds.pad_maskers(f_thr=11000, f_max=1e5)

In [ ]:
#E2=ExcitationPatterns.copyRaw(E)  #no non-maskable part
BW10_0TestFunc=constant_BW10(2400, requires_grad=False)


if io_func=='weibull':
    E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, wb_cdf2)
else:
    E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, sigm)

In [ ]:
#model
u1=u1_mat[0]
pl.figure(figsize=(10,14))
with torch.no_grad():
    ax_list=plotSimulatedCAPs(E2, u1, ylim=[-0.01, 0.01])
    plotSimulatedCAPs(E2, CAParray=vbw_signals_proc, axlist=ax_list)
pl.plot()

In [ ]:
bw_arr=np.linspace(500, 5000, num= ((4000-500)//50+1) )
sigs_ref=vbw_signals_proc
errs=[]
for bw in bw_arr:

    BW10_0TestFunc=constant_BW10(bw, requires_grad=False)


    if io_func=='weibull':
        E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, wb_cdf2)
    else:
        E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, sigm)
    excs = E2.get_tensor() 
    maskingConditions = E2.maskingConditions
    err=0
    for i, exc in zip(range(maskingConditions.n_conditions), excs):
        exc_np = exc.detach().numpy()
        CAP=np.convolve(exc_np, u1, mode='full')
        t=E.t.numpy()
        CAP=CAP[0:len(E2.t)]
        err+=np.sum( (CAP-sigs_ref[i])**2)
    errs.append(err)
    
pl.plot(bw_arr, errs)
pl.xlabel('BW10 model (Hz)')

pl.ylabel('square err')


ind_min=np.argmin(errs)
print(f'estimated bw10: {bw_arr[ind_min]:.0f} Hz')

In [ ]:
#with gradient descent


BW10_0TestFunc=constant_BW10(2000., requires_grad=True)

if io_func=='weibull':
    E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, wb_cdf2)
else:
    E2.set_masking_model(lat_model, BW10_0TestFunc, vbw_maskingConds, sigm)

nb_steps=30
alpha=1e8

alpha_dic[BW10_0TestFunc.BW_10]=alpha
#alpha_dic[E2.E0_maskable]=alpha  #/!| with sum_grad_E0 set to True
    

optim_steps(E2, u1_mat[0], vbw_signals_proc, alpha_dic, 
            nb_steps=nb_steps, #sum_grad_E0=True, 
            #plot_E0_graph=True, plot_masking_I0_graph=True,
            plot_Q10=True, fc_ref_Q10=CF,
           step_plots=5)


8000/BW10_0TestFunc.BW_10

In [ ]:
plotMaskingDegreeFunc(wcdf)

In [ ]:

E2.set_masking_model(lat_model, BW10_0TestFunc, ntch_maskingConds, wcdf)

Reestimation I/O curve (old code)